# Lab 3: Orchestrator-Worker Pattern

This notebook implements the **Orchestrator-Worker Pattern** with **Parallel Execution**:

1. **Orchestrator LLM**: Decides which worker models to call based on the question
2. **Worker Models**: Selected models run **simultaneously** in parallel using async/await
3. **Synthesizer LLM**: Aggregates outputs, synthesizes a final answer, and ranks the workers

## Pattern Flow

```
User Question → Orchestrator LLM ("Which workers to use?")
 ↓
Parallel Worker API Calls (Only selected models)
 ↓
Synthesizer LLM (Merge + Rank)
 ↓
Final Answer + Worker Rankings
```

The **LLM orchestrator** replaces hardcoded model selection with intelligent routing.

In [ ]:
# Start with imports - ask ChatGPT to explain any package you don't know

import os
import json
import asyncio
import random
from datetime import datetime, timedelta
from typing import Dict, List, Any, Tuple
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display
import textwrap

In [ ]:
# Always remember to do this!
load_dotenv(override=True)

In [ ]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

## Step 1: Generate Evaluation Question

First, generate a challenging question to test all the worker models.

In [ ]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [ ]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)

## Step 2: Prepare Worker Configurations

Prepare all available **worker** model configurations that the orchestrator can choose from.

In [ ]:
# ==========================================
# Worker Preparation: All Available Models
# ==========================================

def evaluator_prepare_configs():
    """
    Evaluator: Gathers API keys and prepares configurations for all models.
    Returns a list of model configurations ready for parallel execution.
    """
    configs = []
    
    # Model 1: OpenAI
    configs.append({
        "model_name": "gpt-5-nano",
        "provider": "openai",
        "client": OpenAI(),
        "call_type": "chat.completions",
        "extra_params": {}
    })
    
    # Model 2: Anthropic
    configs.append({
        "model_name": "claude-sonnet-4-5",
        "provider": "anthropic",
        "client": Anthropic(),
        "call_type": "messages.create",
        "extra_params": {"max_tokens": 1000}
    })
    
    # Model 3: Gemini
    configs.append({
        "model_name": "gemini-2.5-flash",
        "provider": "gemini",
        "client": OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/"),
        "call_type": "chat.completions",
        "extra_params": {}
    })
    
    # Model 4: DeepSeek
    configs.append({
        "model_name": "deepseek/deepseek-r1-0528:free",
        "provider": "deepseek",
        "client": OpenAI(api_key=deepseek_api_key, base_url="https://openrouter.ai/api/v1"),
        "call_type": "chat.completions",
        "extra_params": {}
    })
    
    # Model 5: Groq
    configs.append({
        "model_name": "openai/gpt-oss-120b",
        "provider": "groq",
        "client": OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1"),
        "call_type": "chat.completions",
        "extra_params": {}
    })
    
    # Model 6: Ollama (if available)
    configs.append({
        "model_name": "llama3.2",
        "provider": "ollama",
        "client": OpenAI(base_url='http://localhost:11434/v1', api_key='ollama'),
        "call_type": "chat.completions",
        "extra_params": {}
    })
    
    print(f"✅ Evaluator prepared {len(configs)} model configurations")
    return configs

# Prepare global config dictionaries
MODEL_CONFIGS = evaluator_prepare_configs()
MODEL_CONFIGS_BY_NAME = {cfg["model_name"]: cfg for cfg in MODEL_CONFIGS}

## Step 3: Orchestrator LLM - Decide Which Workers to Use

The **orchestrator LLM** analyzes the question and selects which worker models to invoke in parallel.

In [ ]:
# ==========================================
# Orchestrator: LLM Decides Worker Selection
# ==========================================

def list_worker_tools():
    """Return tool descriptions for the orchestrator LLM."""
    tools = []
    for cfg in MODEL_CONFIGS:
        tools.append({
            "name": cfg["model_name"],
            "description": f"Call the {cfg['provider']} model {cfg['model_name']} "
                          "to answer the user question.",
            "parameters": {
                "type": "object",
                "properties": {
                    "reason": {
                        "type": "string",
                        "description": "Why this model is useful for this query."
                    }
                },
                "required": ["reason"]
            }
        })
    return tools

def build_orchestrator_prompt(user_question: str, tools: list) -> list:
    tool_descriptions = "\n".join(
        f"- {t['name']}: {t['description']}" for t in tools
    )

    system_msg = textwrap.dedent(f"""
        You are an **orchestrator** LLM in an orchestrator–worker system.

        You do NOT answer the user's question directly.
        Instead, you decide which worker models to call in parallel.

        Available workers:
        {tool_descriptions}

        Output STRICTLY valid JSON:
        {{
          "models_to_call": [
            {{"name": "<model_name>", "reason": "<short reason>"}},
            ...
          ]
        }}

        Requirements:
        - Choose at least 3 and at most 6 models.
        - Use model names exactly as listed.
        - Prefer diversity (different providers) for hard reasoning tasks.
        - Do not include any fields other than "models_to_call".
    """)

    return [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_question},
    ]

def orchestrator_plan(user_question: str) -> list:
    """
    Ask the orchestrator LLM which workers to use.
    Returns list of selected model names.
    """
    tools = list_worker_tools()
    messages = build_orchestrator_prompt(user_question, tools)

    openai = OpenAI()
    resp = openai.chat.completions.create(
        model="gpt-4o-mini",
        # model="gemini-1.5-flash",
        messages=messages,
        temperature=0.2,
    )
    content = resp.choices[0].message.content
    plan = json.loads(content)
    selected = [m["name"] for m in plan.get("models_to_call", [])]
    print(f"🎯 Orchestrator selected {len(selected)} workers: {selected}")
    return selected

## Step 4: Parallel Worker Execution

Execute **only the selected workers** simultaneously using async/await.

In [ ]:
# ==========================================
# Worker Execution: Async Single Model Call
# ==========================================

async def call_model_async(config: Dict[str, Any], messages: List[Dict]) -> Tuple[str, str]:
    """
    Call a single worker model asynchronously.
    Returns (model_name, answer) or (model_name, error_message).
    """
    model_name = config["model_name"]
    provider = config["provider"]
    client = config["client"]
    call_type = config["call_type"]
    extra_params = config["extra_params"]
    
    try:
        if provider == "anthropic":
            # Anthropic uses a different API structure
            response = await asyncio.to_thread(
                client.messages.create,
                model=model_name,
                messages=messages,
                **extra_params
            )
            answer = response.content[0].text
        else:
            # OpenAI-compatible APIs
            response = await asyncio.to_thread(
                client.chat.completions.create,
                model=model_name,
                messages=messages,
                **extra_params
            )
            answer = response.choices[0].message.content
        
        print(f"✅ {model_name} completed")
        return model_name, answer
        
    except Exception as e:
        error_msg = f"Error calling {model_name}: {str(e)}"
        print(f"❌ {error_msg}")
        return model_name, error_msg

In [ ]:
# ==========================================
# Parallel Worker Execution
# ==========================================

def format_bytes(size: int) -> str:
    """Format bytes into a human-readable string (B, KB, MB)."""
    for unit in ['B', 'KB', 'MB']:
        if size < 1024.0:
            return f"{size:.2f} {unit}"
        size /= 1024.0
    return f"{size:.2f} GB"

async def execute_models_in_parallel(configs: List[Dict[str, Any]], messages: List[Dict]) -> Tuple[List[str], List[str]]:
    """
    Execute selected worker models in parallel.
    """
    print(f"\n🚀 Starting parallel execution of {len(configs)} selected workers...\n")
    
    table_rows = []
    competitors = []
    answers = []
    
    async def call_with_metrics(config):
        model_name = config.get("model_name", "Unknown")
        start_time = datetime.now()
        
        try:
            _, answer = await call_model_async(config, messages)
            end_time = datetime.now()
            
            if isinstance(answer, str) and answer.startswith("Error"):
                status = "❌ Error"
                out_size = 0
            else:
                status = "✅ Success"
                out_size = len(str(answer).encode('utf-8'))
                
        except Exception as e:
            end_time = datetime.now()
            status = "❌ Error"
            answer = str(e)
            out_size = 0

        # Calculate duration
        duration = end_time - start_time
        total_seconds = int(duration.total_seconds())
        mm, ss = divmod(total_seconds, 60)
        hh, mm = divmod(mm, 60)
        dur_str = f"{hh:02d}:{mm:02d}:{ss:02d}" if hh > 0 else f"{mm:02d}:{ss:02d}"

        # Store metrics for table
        table_rows.append({
            "model": model_name,
            "status": status,
            "start": start_time.strftime("%H:%M:%S"),
            "end": end_time.strftime("%H:%M:%S"),
            "duration": dur_str,
            "size": format_bytes(out_size)
        })
        
        return model_name, answer, status

    # Run tasks in parallel
    tasks = [call_with_metrics(config) for config in configs]
    results = await asyncio.gather(*tasks)

    # Process final lists
    for model_name, answer, status in results:
        if status == "✅ Success":
            competitors.append(model_name)
            answers.append(answer)

    # Print Tabular Output
    header = f"{'Model':<25} {'Status':<10} {'Start':<10} {'End':<10} {'Duration':<10} {'Size':<12}"
    print(header)
    print("-" * len(header))
    for row in table_rows:
        print(f"{row['model']:<25} {row['status']:<10} {row['start']:<10} {row['end']:<10} {row['duration']:<10} {row['size']:<12}")
    
    print(f"\n✅ Completed. {len(competitors)}/{len(configs)} workers successful.")
    return competitors, answers

async def mock_execute_models_in_parallel(configs: List[Dict[str, Any]]) -> Tuple[List[str], List[str]]:
    """
    Mocks parallel API calls to display timing and size metrics in a table.
    No actual API calls are made.
    """
    print(f"\n🚀 Starting MOCK execution of {len(configs)} models...\n")
    
    table_rows = []
    competitors = []
    answers = []

    async def mock_api_call(config):
        model_name = config.get("model_name", "Unknown-Model")
        start_time = datetime.now()
        
        # Simulate varying network latency (0.5 to 2.5 seconds)
        await asyncio.sleep(random.uniform(0.5, 2.5))
        
        # Randomly decide if this mock call "fails" (10% chance)
        is_success = random.random() > 0.1
        
        if is_success:
            status = "✅ Success"
            # Mock a response string of random length
            mock_answer = "Mock response data " * random.randint(5, 500)
            out_size = len(mock_answer.encode('utf-8'))
        else:
            status = "❌ Error"
            mock_answer = "Error: Mocked API failure"
            out_size = 0
            
        end_time = datetime.now()
        
        # Calculate duration in mm:ss or hh:mm:ss
        duration = end_time - start_time
        total_seconds = int(duration.total_seconds())
        mm, ss = divmod(total_seconds, 60)
        hh, mm = divmod(mm, 60)
        dur_str = f"{hh:02d}:{mm:02d}:{ss:02d}" if hh > 0 else f"{mm:02d}:{ss:02d}"

        # Record metrics for the final table
        metrics = {
            "model": model_name,
            "status": status,
            "start": start_time.strftime("%H:%M:%S"),
            "end": end_time.strftime("%H:%M:%S"),
            "duration": dur_str,
            "size": format_bytes(out_size)
        }
        
        return model_name, mock_answer, status, metrics

    # Execute mock tasks in parallel
    tasks = [mock_api_call(config) for config in configs]
    results = await asyncio.gather(*tasks)

    # Prepare table headers
    header = f"{'Model':<20} {'Status':<10} {'Start':<10} {'End':<10} {'Duration':<10} {'Size':<12}"
    print(header)
    print("-" * len(header))

    # Output rows and collect final success data
    for model_name, answer, status, row in results:
        print(f"{row['model']:<20} {row['status']:<10} {row['start']:<10} {row['end']:<10} {row['duration']:<10} {row['size']:<12}")
        if status == "✅ Success":
            competitors.append(model_name)
            answers.append(answer)

    print(f"\n✅ Completed. {len(competitors)}/{len(configs)} models simulated successfully.")
    return competitors, answers

async def execute_selected_models(model_names: list, messages: list):
    """Execute only the orchestrator-selected workers."""
    selected_configs = [MODEL_CONFIGS_BY_NAME[m] for m in model_names]
    return await execute_models_in_parallel(selected_configs, messages)

async def mock_execute_selected_models(model_names: list):
    """Execute only the orchestrator-selected workers."""
    selected_configs = [MODEL_CONFIGS_BY_NAME[m] for m in model_names]
    return await mock_execute_models_in_parallel(selected_configs)



## Step 5: Run Orchestrated Pipeline

**Full end-to-end execution**: Orchestrator → Workers → Synthesizer.

In [ ]:
# ==========================================
# FULL ORCHESTRATED PIPELINE
# ==========================================

user_question = question  # From Step 1
messages = [{"role": "user", "content": user_question}]

# 1) Orchestrator chooses workers
selected_models = orchestrator_plan(user_question)

In [ ]:
# 2) Mock execute chosen workers
competitors, answers = await mock_execute_selected_models(selected_models)

In [ ]:
# 2) Run chosen workers in parallel
competitors, answers = await execute_selected_models(selected_models, messages)

## Step 6: Synthesizer LLM - Merge + Rank

The **synthesizer LLM** creates a final answer and ranks the worker models.

In [ ]:
# ==========================================
# Synthesizer: Merge Outputs + Rank Workers
# ==========================================

def aggregator_format_outputs(competitors: List[str], answers: List[str]) -> str:
    """Format worker outputs for the synthesizer."""
    together = ""
    for index, answer in enumerate(answers):
        together += f"# Response from {competitors[index]}\n\n"
        together += answer + "\n\n"
    return together

def build_synthesizer_prompt(question: str, competitors: list, answers: list) -> list:
    formatted = aggregator_format_outputs(competitors, answers)
    num_workers = len(competitors)
    
    system_msg = f"""
    You are evaluating EXACTLY {num_workers} worker responses.
    
    CRITICAL: There are ONLY {num_workers} competitors numbered 1-{num_workers}.
    
    Tasks:
    1. Synthesize final answer from these {num_workers} responses
    2. Rank ONLY these {num_workers} responses (indices 1-{num_workers})
    
    Output STRICTLY valid JSON:
    {{
      "final_answer": "your synthesized answer",
      "rankings": [
        {{"competitor_index": N, "reason": "why N is good"}}  // N is 1-{num_workers}
        // EXACTLY {num_workers} entries, no more, no less
      ]
    }}
    
    Responses ({num_workers} total):
    {formatted}
    """
    
    return [{"role": "system", "content": system_msg}]

def run_synthesizer(question: str, competitors: list, answers: list):
    """Run the synthesizer LLM."""
    messages = build_synthesizer_prompt(question, competitors, answers)
    openai = OpenAI()
    resp = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.3,
    )
    data = json.loads(resp.choices[0].message.content)
    return data

In [ ]:
# ==========================================
# FINAL SYNTHESIS AND RANKINGS
# ==========================================

# Run synthesizer
synth = run_synthesizer(question, competitors, answers)


In [ ]:
for idx, r in enumerate(synth["rankings"], start=1):
    name = competitors[r["competitor_index"] - 1]
    print(f"{idx}. {name} — {r['reason']}")

print(f"\n✅ Pipeline complete! Orchestrator → {len(selected_models)} Workers → Synthesizer")


In [ ]:
# IMMEDIATE DEBUG
print("COMPETITORS:", competitors)
print("NUM COMPETITORS:", len(competitors))
print("RAW SYNTH:", json.dumps(synth, indent=2))
print("RANKINGS:", [r.get("competitor_index") for r in synth.get("rankings", [])])

